#### 상대모멘텀
- 10개의 종목을 이용하여 투자 전략 시스템

    1. 월초부터 월말까지의 수정종가를 이용하여 수익율 생성
    2. 월별 수익율이 높은 n개를 선택
    3. 해당하는 종목을 매수하여 해당하는 종목에 신호가 없어지면 매도하는 형식으로 수익율 생성

In [1]:
import pandas as pd
import numpy as np
import os 
from glob import glob
import warnings

In [2]:
# warning 메시지 출력 방지 
warnings.filterwarnings('ignore')

In [8]:
# 월별 수익율을 계산하는 함수 
def create_1m_rtn(_df, 
                  _ticker, 
                  _start = '2010-01-01', 
                  _col = 'Adj Close'):
    # 복사본 생성 
    result = _df.copy()
    # Date 컬럼이 존재하면 index로 변경 
    if 'Date' in result.columns:
        result.set_index('Date', inplace=True)
    # index를 시계열로 변경 
    result.index = pd.to_datetime(result.index)
    # 시작 시간과 기준이 되는 컬럼으로 필터링 
    result = result.loc[_start: , [_col]]
    # 기준 년월 컬럼을 생성 
    result['STD-YM'] = result.index.strftime('%Y-%m')
    result['1m_rtn'] = 0
    result['CODE'] = _ticker
    # 기준년월(STD-YM)의 유니크한 리스트를 생성
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [9]:
aapl = pd.read_csv("../../csv/AAPL.csv")

In [10]:
sample_aapl, ym_list = create_1m_rtn(aapl, 'AAPL')

In [17]:
sample_aapl.loc[
    sample_aapl['STD-YM'] == '2010-01'
]

,Adj Close,STD-YM,1m_rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL
2010-01-06,26.402260,2010-01,0,AAPL
2010-01-07,26.353460,2010-01,0,AAPL
2010-01-08,26.528664,2010-01,0,AAPL
2010-01-11,26.294640,2010-01,0,AAPL
2010-01-12,25.995537,2010-01,0,AAPL
2010-01-13,26.362217,2010-01,0,AAPL
2010-01-14,26.209538,2010-01,0,AAPL


In [ ]:
ym_list

In [ ]:
# listdir() 파일의 이름의 목록
os.listdir('./data')
# glob() 파일의 경로와 이름의 목록록
files = glob('./data/*.csv')

In [19]:
# 반복문을 이용

# 종복별 데이터프레임 생성

# 월말의 데이터를 모아주는 작업 
# 월별 수익율 계산
stock_df = pd.DataFrame()
month_last_df = pd.DataFrame()

for file in files:
    # file : 주식데이터의 경로와 파일명
    # 경로와 파일명을 나눠준다. 
    folder, name = os.path.split(file)
    # name : AAPL.csv --> AAPL    .csv 나눈다. 
    head, tail = os.path.splitext(name)
    # head 변수는 create_1m_rtn() 함수에서 _ticker 사용

    # csv 파일을 로드 
    read_df = pd.read_csv(file)

    # create_1m_rtn() 함수를 호출
    price_df, ym_list = create_1m_rtn(read_df, _ticker = head)

    # 두번째 반복문 생성
    # price_df의 월별 수익율 계산하는 반복문
    for ym in ym_list:
        # ym : 기준년월
        flag = price_df['STD-YM'] == ym
        # 월초 수정 종가
        buy = price_df.loc[flag].iloc[0, 0]
        # 월말 수정 종가
        sell = price_df.loc[flag].iloc[-1, 0]
        # 수익율 계산
        m_rtn = sell / buy
        # price_df에 flag 조건과 1m_rtn 컬럼에 대입
        price_df.loc[flag, '1m_rtn'] = m_rtn
        # 월말의 데이터를 month_last_df에 단순 행 결합
        last_data = price_df.loc[flag, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat( [month_last_df, last_data] )

In [ ]:
# 데이터 백업
month_rtn_df =  month_last_df.copy()

In [23]:
# 인덱스를 리셋
month_rtn_df.reset_index(inplace=True)

In [25]:
# pivot_table을 이용하여 테이블을 재구조화
month_rtn_df = month_rtn_df.pivot_table(
    index = 'Date', 
    columns= 'CODE', 
    values= '1m_rtn'
)